# MF

In [1]:
# 加载依赖
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from utils.movielens_dataset import MovieLensDatasetWithTorch
from utils.metric import BaseMetric
from utils.trainer import Trainer

In [2]:
# 使用的超参数
config = {
    'TRAIN_BATCH_SIZE': 8,
    'VALID_BATCH_SIZE': 4,
    'TEST_BATCH_SIZE': 4,
    'DATASET_RATIO': [0.8, 0.1, 0.1],
    'DEVICE': 'cpu',
    'NUM_WORKERS': 0,
    'EPOCH': 100,
    'NUM_FEATURE': 111,
    'POS_WEIGHT': 2,
    'LEARNING_RATE': 1e-3,
    'K': 8,
}

In [3]:
# 加载数据集
# MovieLens
with open('../dataset/ml-latest-small-ratings.txt', 'r', encoding='utf-8') as f:
    ml_dataset = MovieLensDatasetWithTorch(f, task='rating')
dataset_size = len(ml_dataset)
dataset_split_size = [int(dataset_size * r) for r in config['DATASET_RATIO']]
if sum(dataset_split_size) != dataset_size:
    dataset_split_size[-1] += dataset_size - sum(dataset_split_size)
train_set, valid_set, test_set = torch.utils.data.random_split(ml_dataset, dataset_split_size)

user_num, item_num = ml_dataset.user_num, ml_dataset.item_num

In [4]:
type(ml_dataset)

utils.movielens_dataset.MovieLensDatasetWithTorch

In [5]:
train_loader = DataLoader(
    dataset=train_set,
    batch_size=config['TRAIN_BATCH_SIZE'],
    shuffle=True,
    num_workers=config['NUM_WORKERS']
)

valid_loader = DataLoader(
    dataset=valid_set,
    batch_size=config['VALID_BATCH_SIZE'],
    shuffle=False,
    num_workers=config['NUM_WORKERS']
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=config['TEST_BATCH_SIZE'],
    shuffle=False,
    num_workers=config['NUM_WORKERS']
)

In [6]:
# 构建模型
class MF(nn.Module):
    def __init__(self, user_num, item_num, k):
        super(MF, self).__init__()
        U = torch.zeros((user_num, k))
        I = torch.zeros((item_num, k))
        user_bias = torch.zeros((user_num, k))
        item_bias = torch.zeros((item_num, k))

        nn.init.xavier_uniform_(U)
        nn.init.xavier_uniform_(I)
        nn.init.xavier_uniform_(user_bias)
        nn.init.xavier_uniform_(item_bias)

        self.U = nn.Parameter(U, requires_grad=True)
        self.I = nn.Parameter(I, requires_grad=True)
        self.user_bias = nn.Parameter(user_bias, requires_grad=True)
        self.item_bias = nn.Parameter(item_bias, requires_grad=True)

    def forward(self, batch_x):
        batch_x = batch_x[0]
        idxs = batch_x.coalesce().indices()
        user_idx, item_idx = idxs[0, :], idxs[1, :]
        user_factor = self.U[user_idx, :]
        item_factor = self.I[item_idx, :]
        user_bias = self.user_bias[user_idx, :]
        item_bias = self.item_bias[item_idx, :]
        # print((user_factor * item_factor).shape)
        return torch.sum(user_factor * item_factor + user_bias + item_bias, dim=1)

    def rec(self, k=10):
        return torch.topk(torch.matmul(self.U, self.I.t()), k=k, dim=1)

In [7]:
model = MF(user_num, item_num, k=config['K'])
optimizer = optim.Adam(lr=config['LEARNING_RATE'], params=model.parameters())
loss_func = nn.MSELoss()
metric = BaseMetric()

trainer = Trainer(
    model=model,
    loss_func=loss_func,
    optimizer=optimizer,
    metric=metric,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    config=config,
)

==========Config==========
TRAIN_BATCH_SIZE: 8
VALID_BATCH_SIZE: 4
TEST_BATCH_SIZE: 4
DATASET_RATIO: [0.8, 0.1, 0.1]
DEVICE: cpu
NUM_WORKERS: 0
EPOCH: 100
NUM_FEATURE: 111
POS_WEIGHT: 2
LEARNING_RATE: 0.001
K: 8


In [8]:
if __name__ == '__main__':
    trainer.train()
    trainer.test()

==========TRAIN BEGIN==========


100%|██████████| 61/61 [00:00<00:00, 121.12it/s]


Train Epoch: 1
Loss: 11.68290144498231


100%|██████████| 16/16 [00:00<00:00, 1206.47it/s]


Valid Epoch: 1
loss: 9.967808306217194



...

Train Epoch: 100
Loss: 0.8146616191160484


100%|██████████| 16/16 [00:00<00:00, 1198.33it/s]


Valid Epoch: 100
loss: 0.9368579536676407

==========TRAIN END==========


100%|██████████| 16/16 [00:00<00:00, 1148.85it/s]

Test Loss: 1.1258496642112732
loss: 1.1258496642112732



In [11]:
model.rec()[1][0]

tensor([   99,  6460,    53, 27751,   496,  3567,  2969,   148, 26326,  6818])